# [ TD(Temporal Difference)와 Action의 탐색 ]

DP의 벨만방정식과 MC의 평균증분을 도입한 model-free방법(환경의 확률구조를 모른다고 가정)


-------




## 1. SARSA
- **On-policy 학습** : action at 선택을 위해 epsilon-greedy policy 사용

- a(t+1)의 선택에도 epsilon-greedy policy를 사용(동일한 policy 업데이트 전략을 사용함)

In [1]:
import gym
import random
env=gym.make('FrozenLake-v0')

Q={}
for s in range(env.observation_space.n):
    for a in range(env.action_space.n):
        Q[(s,a)]=random.uniform(0,1)  # 초기치 지정

In [2]:
# 입실론 greedy 알고리즘 지정

def epsilon_greedy(state,epsilon):
    if random.uniform(0,1)<epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)),key=lambda x:Q[(state,x)])

In [3]:
alpha=0.1
gamma=0.9
epsilon=0.2   # 입실론 0.2로 설정

num_episodes=200000  # 에피소드 수
timesteps=1000

for i in range(num_episodes):
    s=env.reset()
    a=epsilon_greedy(s,epsilon)  # action을 입실론 greedy로 생성
    
    for t in range(timesteps):
        s_,r,done,_ =env.step(a)
        a_=epsilon_greedy(s_,epsilon)
        
        Q[(s,a)]=Q[(s,a)]+alpha*(r+gamma*Q[(s_,a_)]-Q[(s,a)])
        s=s_
        a=a_
        if done:
            break       


In [5]:
import pandas as pd
df=pd.DataFrame(list(Q.items()), columns=['state_action_pair', 'value'])
print(df.head(12))   # state, action별 value값 (가장 큰 것이 optimal policy)

   state_action_pair     value
0             (0, 0)  0.470423
1             (0, 1)  0.489406
2             (0, 2)  0.490700
3             (0, 3)  0.470153
4             (1, 0)  0.544379
5             (1, 1)  0.502286
6             (1, 2)  0.585874
7             (1, 3)  0.492480
8             (2, 0)  0.528196
9             (2, 1)  0.556400
10            (2, 2)  0.540419
11            (2, 3)  0.516060


In [7]:
df.head(12)  # state, action별 value : 가장 높은 값을 최적의 policy로 선택!

,state_action_pair,value
0,"(0, 0)",0.470423
1,"(0, 1)",0.489406
2,"(0, 2)",0.490700
3,"(0, 3)",0.470153
4,"(1, 0)",0.544379
5,"(1, 1)",0.502286
6,"(1, 2)",0.585874
7,"(1, 3)",0.492480
8,"(2, 0)",0.528196
9,"(2, 1)",0.556400


## 2. Q - 학습 
- **Off-policy 학습** : a(t)의 선택은 epsilon-greedy policy를 사용
- a(t+1)의 선택에는 max Q(s(t+1), a')를 사용함 
> max 값을 사용하기에 확률변수 Q(s, a)를 과대 추정할 위험이 있음. 따라서 실제로 SARSA보다 value값이 높게 나타남

In [8]:
import gym
import random
import numpy as np
env=gym.make('FrozenLake-v0')

Q={}
for s in range(env.observation_space.n):
    for a in range(env.action_space.n):
        Q[(s,a)]=random.uniform(0,1)  # 초기치 지정(uniform으로)

In [9]:
def epsilon_greedy(state,epsilon):
    if random.uniform(0,1)<epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)),key=lambda x:Q[(state,x)])

In [10]:
alpha=0.1
gamma=0.9
epsilon=0.2

num_episodes=200000  # 에피소드 수
timesteps=1000

for i in range(num_episodes):
    s=env.reset()
    
    for t in range(timesteps):
        a=epsilon_greedy(s,epsilon)
        s_,r,done,_ =env.step(a)
        a_=np.argmax([Q[(s_,a)] for a in range(env.action_space.n)])
        
        Q[(s,a)]=Q[(s,a)]+alpha*(r+gamma*Q[(s_,a_)]-Q[(s,a)])
        s=s_
        if done:
            break       


In [11]:
import pandas as pd
df=pd.DataFrame(list(Q.items()), columns=['state_action_pair', 'value'])
print(df.head(12))

   state_action_pair     value
0             (0, 0)  0.536820
1             (0, 1)  0.566512
2             (0, 2)  0.590190
3             (0, 3)  0.533457
4             (1, 0)  0.592051
5             (1, 1)  0.619920
6             (1, 2)  0.631278
7             (1, 3)  0.570629
8             (2, 0)  0.617782
9             (2, 1)  0.616845
10            (2, 2)  0.689648
11            (2, 3)  0.607124


In [12]:
df.head(12)

,state_action_pair,value
0,"(0, 0)",0.536820
1,"(0, 1)",0.566512
2,"(0, 2)",0.590190
3,"(0, 3)",0.533457
4,"(1, 0)",0.592051
5,"(1, 1)",0.619920
6,"(1, 2)",0.631278
7,"(1, 3)",0.570629
8,"(2, 0)",0.617782
9,"(2, 1)",0.616845
